In [101]:
%display latex
from pseries_basis import *
from ore_algebra import *
n = PSBasis.n(PSBasis)

In [102]:
B = BinomialBasis(); B

Binomial basis (x) choose n

In [103]:
OE.<E> = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0))
x = OE.base().gens()[0]

In [161]:
def seq_from_operator(operator, seq):
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    print(coefficient)
    return lambda i : sum(coefficients[j](v=i)*seq(i+j) for j in range(len(coefficients)))

def eval_ore_operator(operator, ring=None,**values):
    gens = [str(el) for el in operator.parent().gens()]
    outer_vals = {el : values.get(el, 0) for el in gens}
    inner_vals = {el : values[el] for el in values if (not el in outer_vals)}
    coefficients = [el(**inner_vals) for el in operator.polynomial().coefficients()]
    monomials = [prod(
        outer_vals[str(g)]**(m.degree(g)) for g in operator.polynomial().parent().gens()
    ) for m in operator.polynomial().monomials()]
    result = sum(coefficients[i]*monomials[i] for i in range(len(monomials)))
    if ring != None:
        return ring(result)
    return result

def print_recurrence(operator):
    if len(operator.parent().gens()) > 1:
        raise TypeError()
    E = operator.parent().gens()[0]
    v = operator.parent().base().gens()[0]
    coefficients = operator.coefficients(sparse=False)
    
    return " + ".join(
        "(" + 
        str(coefficients[j]).replace(str(v),'n') + 
        ")*"
        "a_{n+" + str(j) + "}" for j in range(len(coefficients))) + " = 0" 

In [162]:
L = (x+1)*E + 2*(3 - 2*x)
L1=eval_ore_operator(B.recurrence(L), L.parent(),Sn=E, n=x, Sni=1)
L2=eval_ore_operator(B.recurrence(L1), L1.parent(),Sn=E, n=x, Sni=1)
L2

(x + 1)*E - 7*x + 8

In [165]:
X = lambda x : 1
f = seq_from_operator(L, X)

In [166]:
[f(i) for i in range(10)]

[-3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7,
 -3*x + 7]

In [151]:
str(L).replace('x', 'n')

'(n + 1)*E^2 - 4*n + 6'

In [134]:
sol = lambda x : (7^(x-1) * rising_factorial(-1/7, x-1))/factorial(x)

In [135]:
f = lambda x : catalan_number(x)
g = lambda x : sum(sum(binomial(x,k)*binomial(k,m)*sol(m) for m in range(k+1)) for k in range(x+1))

In [136]:
[f(i) for i in range(10)]

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

In [137]:
[g(i).simplify_full() for i in range(10)]

[-1/8, 3/4, 3, 7, 27/4, -99/2, -1011/2, -24984/7, -1328013/56, -4450861/28]

In [139]:
[((i+1)*g(i+1) + 2*(3-2*i)*g(i)).simplify_full() for i in range(10)]

[0, 15/2, 15, -15, -315, -2340, -15885, -111195, -5698395/7, -86417595/14]